## Basics of TS

이 커널을 시계열 데이터를 활용하도록 만들어졌다.

## Table of contents

- Competition and data overview
- Imports ( data and packages )
- Basic exploration/EDA
- Single time-series
    - Stationarity
    - Seasonality , Trend and Remainder
    - AR , MA , ARMA , ARIMA
    - Selecting P and Q using AIC
    - ETS
    - Prophet
    - UCM
- Hierarchical time-series
    - Bottom's up
    - AHP
    - PHA
    - FP

## Competition and data overview:

- IC 회사는 어떤 일을 하는가?  
    Enterprise 8 system of programs은 관리 회계, 비즈니스 회계, HR 관리, CRM, SRM, MRP, MRP 등과 같은 경제 및 관리 활동의 다양한 비즈니스 작업과 같은 일상적인 기업 활동을 자동화 하는 일을 한다.  
    
    
- 데이터  
    각 지점-물건의 조합에 대한 일일 판매 데이터가 제공되지만, 우리는 **매월** 판매랑을 예측해야된다.

## Imports ( data and packages )